## Imports And Some Settings

In [35]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True))
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from sklearn.datasets import load_boston
tf.keras.backend.set_floatx('float64')

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 980, pci bus id: 0000:26:00.0, compute capability: 5.2
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



## Loading Data With sklearn. Creating Train, Val And Test Datasets

In [2]:
boston_dataset = load_boston()
boston_keys = boston_dataset.keys()

X_dataset = boston_dataset['data']
y_dataset = boston_dataset['target']

border_1 = int(X_dataset.shape[0] * 0.7)
border_2 = int(border_1 + X_dataset.shape[0] * 0.2)

print("Train dataset size:",border_1)
print("Validation dataset size:",border_2 - border_1)
print("Test dataset size:",X_dataset.shape[0] - border_2)

Train dataset size: 354
Validation dataset size: 101
Test dataset size: 51


In [14]:
X_train = X_dataset[:border_1]
y_train = y_dataset[:border_1]
X_val = X_dataset[border_1:border_2]
y_val = y_dataset[border_1:border_2]
X_test = X_dataset[border_2:]
y_test = y_dataset[border_2:]

## Make pd.dataFrames From nd.arrays

In [15]:
def ndarray_to_dataFrame(data, labels=None):
    index = np.arange(0,data.shape[0],1)
    return pd.DataFrame(data=data, index=index, columns=labels)

In [16]:
x_columns_names = boston_dataset['feature_names']
y_column_name = np.array(['PRICE'])
X_train = ndarray_to_dataFrame(X_train, x_columns_names)
y_train = ndarray_to_dataFrame(y_train)
X_val = ndarray_to_dataFrame(X_val, x_columns_names)
y_val = ndarray_to_dataFrame(y_val, y_column_name)
X_test = ndarray_to_dataFrame(X_test, x_columns_names)
y_test = ndarray_to_dataFrame(y_test, y_column_name)

## Dataset Preview

In [17]:
pd.DataFrame.join(X_train[:10], y_train[:10])

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,0
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9


## Tensorflow Linear Regression: Implementation

In [18]:
feature_columns = [tf.feature_column.numeric_column(f_name, dtype=tf.float32) for f_name in boston_dataset['feature_names']]

In [52]:
def make_input_fn(data_df, label_df, epochs_number, shuffle=True, batch_size=32):
    def input_fn():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df.squeeze()))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(epochs_number)
        return ds
    return input_fn

In [59]:
train_input_fn = make_input_fn(X_train, y_train, 3)
eval_input_fn = make_input_fn(X_val, y_val, 3)
test_input_fn = make_input_fn(X_test, y_test, 1, shuffle=False, batch_size=1)

## Tensorflow Linear Regression: Model Selection

## Hand-made Linear Regression: Implementation (Normal Eqaution)

## Hand-made Linear Regression: Model Selection

## Comparison Of Tensorflow And Hand-made Implementations: